This notebook is used to inspect embryos after mask was created.  
For each embryo the widget displays the max projection of the fish, dapi, and gfp channels + mask.  
The user selects which embryos are valid and if the mask has to be fixed.  
In the second part of the notebook, the the embryos.csv "status" column will update accordingly, and csv is saved.  

Please GIT COMMIT csv after changes.  

In [1]:
import pandas as pd
from glob import glob
import os
from skimage import io

import ipywidgets as widgets
from functools import partial

import time

from IPython.display import clear_output

In [57]:
csv_path = '/PATH0'
preview_path = 'PATH1'
save_txt_dir = 'PATH2'
preview_png_dir = preview_path

In [58]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_results_txt_path = os.path.join(save_txt_dir, f'embryos_status_updates_{timestr}.txt')
fix_masks_txt_path = os.path.join(save_txt_dir, f'embryos_masks_to_fix_{timestr}.txt')

#### Read CSV

In [59]:
csv_file = pd.read_csv(csv_path)
csv_file = csv_file.reset_index(drop=True)

# Find all the rows/embryos to predict stage:
#df_embryos_to_predict = csv_file[(csv_file["#nucs_predicted"]==-1) & ((csv_file["status"]==1) | (csv_file["status"]==-1)) & (csv_file["#channels"]>3)]

df_to_view = csv_file[(csv_file["status"]==0) & (~csv_file["cropped_image_file"].isna()) & (csv_file["#channels"]>3)]

#### Check in all files have previews:

In [60]:
all_preview_paths = glob(os.path.join(preview_path,'*.png'))
all_preview_names = [os.path.basename(f)[:-4] for f in all_preview_paths]

In [61]:
df_names = [df_to_view.at[i,"cropped_image_file"][:-4] for i in df_to_view.index]

In [62]:
print('missing files in preview dir:')
set(df_names) - set(all_preview_names)

missing files in preview dir:


set()

#### Create Preview PNG:

In [63]:
# for n in df_names:
#     path_png = os.path.join(preview_png_dir,f'{n[:-4]}.png')
#     if not os.path.exists(path_png):
#         im = io.imread(os.path.join(preview_path,n))
#         io.imsave(path_png, im)

### Show Preview:

In [64]:
def on_button_clicked(curr_img, texts, buttons, buttons1, b):
    
    last_beg = curr_img-images_per_page
        
    with open(save_results_txt_path,"a") as f:
        f.write("\n")
        f.write("\n".join([f'{texts[i].value},{buttons[i].value}' for i in range(len(buttons))]))
    
    with open(fix_masks_txt_path,"a") as f:
        f.write("\n")
        f.write("\n".join([f'{texts[i].value},{buttons1[i].value}' for i in range(len(buttons1))]))
        
    clear_output()
    
    display(select_images_widget(curr_img))

In [65]:
def select_images_widget(curr_img_num):
    COLS = 5
    ROWS = int(images_per_page/COLS)
    
    IMG_WIDTH = 200
    IMG_HEIGHT = 100
    
    texts = []
    images = []
    buttons = []
    buttons1 = []
    
    rows = []
    
    for row in range(ROWS):
        cols = []
        for col in range(COLS):

            index = (row * COLS + col)
            
            im_name = df_names[index+curr_img_num]
            
            file = open(os.path.join(preview_png_dir,f'{im_name}.png'), "rb")
            image = file.read()

            texts.append(widgets.Text(value=im_name))
            
            images.append(widgets.Image(
                value=image, width=IMG_WIDTH, height=IMG_HEIGHT)
            )

            buttons.append(widgets.ToggleButton(value=True,
                                          description='Use embryo',
                                          disabled=False,
                                          #button_style='', # 'success', 'info', 'warning', 'danger' or ''
                                          tooltip='Description',
                                          icon='check'))
            
            buttons1.append(widgets.ToggleButton(value=False,
                              description='Fix mask?',
                              disabled=False,
                              #button_style='', # 'success', 'info', 'warning', 'danger' or ''
                              tooltip='Description',
                              icon='check'))
            
            box_layout = widgets.Layout(display='flex',
                            flex_flow='column',
                            width='50%')
            
            # Create a vertical layout box, image above the button
            box = widgets.VBox([texts[index], images[index], buttons[index], buttons1[index]], layout=box_layout)
            cols.append(box)

        # Create a horizontal layout box, grouping all the columns together
        rows.append(widgets.HBox(cols))
        
    button = widgets.Button(description="Next..")
    button.on_click(partial(on_button_clicked, curr_img_num+images_per_page, texts, buttons, buttons1))
    rows.append(button)
    
    # Create a vertical layout box, grouping all the rows together
    results = widgets.VBox(rows)
    return results

In [66]:
images_per_page = 20
select_images_widget(0)

### Update embryo.csv from txt

If anything looks fishy at this stage, nothing changes in the txt files or the csv files (until the csv file is saved in the last cell).  
So you can always rerun it from this point when the txt files are loaded.

In [67]:
with open(save_results_txt_path,'r') as f:
    lines = f.read().split('\n')

In [68]:
with open(fix_masks_txt_path,'r') as f:
    lines1 = f.read().split('\n')

In [69]:
lines = list(filter(None, lines))
lines1 = list(filter(None, lines1))

In [70]:
names = [l.split(',')[0] for l in lines]
status = [l.split(',')[1] for l in lines]

In [71]:
names_masks = [l.split(',')[0] for l in lines1]
is_mask_bad = [l.split(',')[1] for l in lines1]

# Take only bad masks
bad_masks = [names_masks[i] for i,boo in enumerate(is_mask_bad) if boo=='True']

In [72]:
len(names_masks),len(bad_masks)

(900, 34)

In [73]:
# Check for doubles:

if len(names) != len(set(names)):
    print('WE GOT DOUBLES!! Taking "status" from last choice only')
else:
    print('Yay! No doubles!!')

Yay! No doubles!!


In [74]:
## Cross with masks list:
## I expect that if "fix mask" was chosen then "status" should be 1 (good embryo)

for n in bad_masks:
    idx = names.index(n)
    if status[idx] == 'False':
        print(f'{n} is in the fix mask list but status is bad. Fixing Status to good')
        status[idx] = 'True'

In [75]:
csv_file = pd.read_csv(csv_path)
csv_file = csv_file.reset_index(drop=True)

In [76]:
## Status - 1=True, 3=False (aka bad embryo)

In [77]:
for i,n in enumerate(names):
    idx = csv_file.index[csv_file['cropped_image_file'] == f'{n}.tif'].tolist()
    if len(idx)!=1:
        print(f'oh oh. found {len(idx)} csv rows for {n}')
    else:
        csv_file.at[idx[0],'status'] = 1 if status[i]=='True' else 3

In [78]:
# Save csv

csv_file.to_csv(csv_path, index=False)

### DONT FORGET TO COMMIT AND PUSH!!
by runnning this in the "embryos_csv" dir:  
```git add embryos.csv  
git commit -m "new embryos and masks verified"  
git push origin master  
```